<a href="https://colab.research.google.com/github/Tomawock/MLDM_COVID-19/blob/master/datasets/dataset_finale_medie_ponderate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creazione di vari Dataset dove son già raggruppate le X e le Y in base alle necessità

In questo dataset le medie sono ponderate (con maggiore importanza per le date più a ritroso nel tempo). Il resto ricalca ciò che avviene in "dataset_finale_medie.ipynb".

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pandas as pd
import os
import random
os.chdir('/content/drive/Shared drives/MLDM progetto/Datasets/dataset_finali')

In [24]:
#Carica dati dai file
dataset_completo = pd.read_csv("dataset_completo.csv")

In [25]:
def media_ponderata(giorni, index_oldest):
  ponderatore = abs(giorni.index-index_oldest-14)
  return giorni.multiply(pd.Series(ponderatore, index=giorni.index), axis="index").sum()/sum(ponderatore)


##Creazione dataset denominato: dataset_finale_medie_ponderate.csv

In questo dataset le X sono composte da tutti i parametri presi sei giorni stocasticamente compresi fra le due settimane precedenti. Di questi dati viene computata la media aritmetica.


In [26]:
dataset_completo

,Data,Ammoniaca,Benzene,Biossido di Azoto,Biossido di Zolfo,BlackCarbon,Monossido di Azoto,Monossido di Carbonio,Ossidi di Azoto,Ozono,PM10 (SM2005),Particelle sospese PM2.5,Altezza Neve,Direzione Vento,Livello Idrometrico,Precipitazione,Radiazione Globale,Temperatura,Umidità Relativa,Velocità Vento,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi
0,2020-02-24,22.366667,0.861111,29.932514,2.997511,1.7,0.841667,0.682971,44.105022,39.832055,48.822581,38.548387,90.537022,184.545614,25.365786,0.000033,150.665814,10.307715,64.635167,2.469884,76,19,95,71,166,0,166,0,6,172,1463
1,2020-02-25,28.266667,1.135000,37.558723,2.720432,2.7,0.337500,0.737955,56.730520,24.368546,66.269841,50.774194,90.435719,181.123030,26.283322,0.000234,49.566798,7.441500,86.661125,1.223718,79,25,104,127,231,65,68,0,9,240,3700
2,2020-02-26,16.588889,0.738095,24.832697,2.235606,1.8,0.100000,0.625595,40.744959,44.143674,31.758065,25.066667,93.372616,232.972627,25.411972,0.000480,122.909575,5.420617,54.402244,3.748763,79,25,104,145,249,18,18,0,9,258,3208
3,2020-02-27,12.077778,0.504545,28.488100,2.571915,0.8,0.145833,0.535147,42.107602,51.350968,13.000000,8.366667,92.302654,211.205210,25.506752,0.000537,116.845181,2.955113,44.979290,2.293375,172,41,213,136,349,100,145,40,14,403,3320
4,2020-02-28,12.355556,0.613043,27.597785,2.379423,0.8,0.125000,0.531727,42.153344,58.694750,15.661290,11.225806,94.503415,203.283491,24.184149,0.000541,176.738199,5.226649,34.199895,2.951837,235,47,282,192,474,125,128,40,17,531,4835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2020-07-28,21.311111,0.304762,19.167302,2.066222,1.6,0.454167,0.405176,24.214943,98.499780,25.836066,18.343750,8.098215,183.136548,26.897610,0.013825,285.910730,23.619529,70.332060,1.760354,151,13,164,6514,6678,-47,53,72528,16802,96008,1271875
156,2020-07-29,18.355556,0.319048,17.284192,1.987500,1.3,0.200000,0.412268,22.116009,97.153486,25.338983,17.612903,7.189119,173.743580,24.699126,0.000836,288.471169,24.344374,70.035225,1.717794,154,13,167,6396,6563,-115,46,72689,16802,96054,1280533
157,2020-07-30,24.177778,0.327273,18.415901,1.956380,1.3,0.356522,0.428788,22.861944,101.361138,28.625000,19.321429,6.915503,183.587786,24.692338,0.017057,264.903506,25.383323,70.221072,1.730722,151,13,164,5871,6035,-528,88,73305,16802,96142,1291740
158,2020-07-31,23.611111,0.336364,20.929353,1.937361,1.4,0.354167,0.413101,26.127682,97.802334,24.278689,17.096774,7.036273,190.575700,24.861249,0.002085,287.864090,25.667782,61.720169,1.797121,148,7,155,5856,6011,-24,77,73402,16806,96219,1300088


In [29]:
random.seed(220)

col=list(dataset_completo.columns)
col.remove("Data")

for i in range(0,dataset_completo.shape[0]-18):
  data_previsione=dataset_completo.loc[i+14, "Data"]
  giorni_precedenti=dataset_completo.loc[i:i+14, :]
  giorni_selezionati=random.sample(range(i,i+14), 6)
  giorni_estratti=dataset_completo.loc[giorni_selezionati, dataset_completo.columns != "Data"]
  media_passato=media_ponderata(giorni_estratti, i).to_frame()
  
  media_passato=media_passato.pivot_table(columns=col).add_prefix("pass_")
  media_passato["Data"]=data_previsione
  fut1=pd.to_numeric(dataset_completo.loc[i+16, dataset_completo.columns != "Data"]).to_frame().pivot_table(columns=col).add_prefix("fut1_").reset_index()
  fut2=pd.to_numeric(dataset_completo.loc[i+18, dataset_completo.columns != "Data"]).to_frame().pivot_table(columns=col).add_prefix("fut2_").reset_index()
  riga=pd.concat([media_passato, fut1, fut2], axis=1).drop(columns="index")
  if i==0:
    finale=pd.DataFrame(columns=riga.columns)
  
  finale=pd.concat([finale, riga], axis=0, ignore_index=True)

colonne_non_ordinate=list(finale.columns)
colonne_non_ordinate.remove("Data")
elenco_ordinato_colonne=["Data"]

elenco_ordinato_colonne=elenco_ordinato_colonne+colonne_non_ordinate
finale=finale.reindex(columns=elenco_ordinato_colonne)
finale

,Data,pass_Altezza Neve,pass_Ammoniaca,pass_Benzene,pass_Biossido di Azoto,pass_Biossido di Zolfo,pass_BlackCarbon,pass_Direzione Vento,pass_Livello Idrometrico,pass_Monossido di Azoto,pass_Monossido di Carbonio,pass_Ossidi di Azoto,pass_Ozono,pass_PM10 (SM2005),pass_Particelle sospese PM2.5,pass_Precipitazione,pass_Radiazione Globale,pass_Temperatura,pass_Umidità Relativa,pass_Velocità Vento,pass_deceduti,pass_dimessi_guariti,pass_isolamento_domiciliare,pass_nuovi_positivi,pass_ricoverati_con_sintomi,pass_tamponi,pass_terapia_intensiva,pass_totale_casi,pass_totale_ospedalizzati,pass_totale_positivi,pass_variazione_totale_positivi,fut1_Altezza Neve,fut1_Ammoniaca,fut1_Benzene,fut1_Biossido di Azoto,fut1_Biossido di Zolfo,fut1_BlackCarbon,fut1_Direzione Vento,fut1_Livello Idrometrico,fut1_Monossido di Azoto,...,fut1_dimessi_guariti,fut1_isolamento_domiciliare,fut1_nuovi_positivi,fut1_ricoverati_con_sintomi,fut1_tamponi,fut1_terapia_intensiva,fut1_totale_casi,fut1_totale_ospedalizzati,fut1_totale_positivi,fut1_variazione_totale_positivi,fut2_Altezza Neve,fut2_Ammoniaca,fut2_Benzene,fut2_Biossido di Azoto,fut2_Biossido di Zolfo,fut2_BlackCarbon,fut2_Direzione Vento,fut2_Livello Idrometrico,fut2_Monossido di Azoto,fut2_Monossido di Carbonio,fut2_Ossidi di Azoto,fut2_Ozono,fut2_PM10 (SM2005),fut2_Particelle sospese PM2.5,fut2_Precipitazione,fut2_Radiazione Globale,fut2_Temperatura,fut2_Umidità Relativa,fut2_Velocità Vento,fut2_deceduti,fut2_dimessi_guariti,fut2_isolamento_domiciliare,fut2_nuovi_positivi,fut2_ricoverati_con_sintomi,fut2_tamponi,fut2_terapia_intensiva,fut2_totale_casi,fut2_totale_ospedalizzati,fut2_totale_positivi,fut2_variazione_totale_positivi
0,2020-03-09,102.693766,12.048366,0.652610,28.006591,2.462092,1.232353,194.823706,29.026687,0.245466,0.570348,42.669083,46.897132,17.719451,13.478994,0.049216,122.921316,3.925344,56.219325,2.534386,46.176471,141.705882,269.147059,242.147059,529.205882,6.797147e+03,113.352941,1099.588235,642.558824,911.705882,190.323529,116.508564,13.275000,0.680952,31.763360,2.866919,1.5,190.502696,28.951725,0.754167,...,900.0,1351.0,1489.0,3852.0,25629.0,560.0,7280.0,4412.0,5763.0,1336.0,109.034233,14.012500,0.533333,20.682788,2.456760,0.7,202.414057,28.240583,0.375000,0.516473,27.530027,49.417757,24.709677,19.966667,0.001532,152.153664,11.042177,74.557500,1.945538,890.0,1198.0,2647.0,1095.0,4435.0,32700.0,650.0,9820.0,5085.0,7732.0,836.0
1,2020-03-10,107.788239,13.570000,0.716374,26.330834,2.409004,1.537500,186.626209,31.600534,0.298438,0.594606,40.127309,42.096282,20.458123,16.654390,0.059424,102.845946,4.433176,69.488646,2.405312,52.200000,146.350000,404.200000,317.825000,609.150000,8.248425e+03,138.275000,1350.175000,747.425000,1151.625000,277.875000,110.362902,17.637500,0.745455,30.971685,2.516601,1.8,204.788320,28.628933,0.362500,...,1085.0,2044.0,1445.0,4247.0,29534.0,605.0,8725.0,4852.0,6896.0,1133.0,112.331299,11.088889,0.595652,14.829834,1.991204,1.0,146.477153,32.057624,0.141667,0.494544,19.485288,46.814770,25.650794,20.806452,0.068590,42.026965,6.675531,86.111540,1.919871,966.0,1660.0,3429.0,1865.0,4898.0,37138.0,732.0,11685.0,5630.0,9059.0,1327.0
2,2020-03-11,109.919061,15.361220,0.784263,31.277777,2.573798,1.858824,177.779410,32.441453,0.255310,0.606111,48.648364,39.648569,21.582479,16.756641,0.048519,109.109624,4.458184,61.068755,2.452192,46.392157,139.784314,360.411765,242.156863,555.000000,8.176314e+03,133.764706,1235.352941,688.764706,1049.176471,199.862745,109.034233,14.012500,0.533333,20.682788,2.456760,0.7,202.414057,28.240583,0.375000,...,1198.0,2647.0,1095.0,4435.0,32700.0,650.0,9820.0,5085.0,7732.0,836.0,112.449581,14.544444,0.560870,16.264336,1.800396,1.8,166.463890,29.875236,0.233333,0.521769,21.749993,42.753186,18.000000,15.333333,0.001614,127.353711,5.350083,79.784170,1.404195,1218.0,2011.0,3776.0,1587.0,5500.0,40369.0,767.0,13272.0,6267.0,10043.0,984.0
3,2020-03-12,98.991683,13.599664,0.612758,26.494288,2.447746,1.221739,201.325837,26.097434,0.276087,0.562087,39.

In [30]:
finale.to_csv("dataset_finale_medie_ponderate_rnd220.csv", index=False)